In [ ]:
from pyincore_viz import PlotUtil
from pyincore import IncoreClient, Dataset
from pyincore.analyses.bridgedamage import BridgeDamage
from pyincore.analyses.meandamage import MeanDamage
import json

### bridge damage analysis

In [ ]:
client = IncoreClient()

# New madrid earthquake using Atkinson Boore 1995
hazard_type = "earthquake"
hazard_id = "5b902cb273c3371e1236b36b"

# NBSR bridges
bridge_dataset_id = "5a284f2dc7d30d13bc082040"

# Default Bridge Fragility Mapping on incore-service
mapping_id = "5b47bcce337d4a37755e0cb2"

# Use hazard uncertainty for computing damage
use_hazard_uncertainty = False
# Use liquefaction (LIQ) column of bridges to modify fragility curve
use_liquefaction = False

# Create bridge damage
bridge_dmg = BridgeDamage(client)
# Load input datasets
bridge_dmg.load_remote_input_dataset("bridges", bridge_dataset_id)
# Specify the result name
result_name = "bridge_result"
# Set analysis parameters
bridge_dmg.set_parameter("result_name", result_name)
bridge_dmg.set_parameter("mapping_id", mapping_id)
bridge_dmg.set_parameter("hazard_type", hazard_type)
bridge_dmg.set_parameter("hazard_id", hazard_id)
bridge_dmg.set_parameter("num_cpu", 4)

# Run bridge damage analysis
bridge_dmg.run_analysis()

### geo map

In [ ]:
bridge_inventory = bridge_dmg.get_input_dataset('bridges')
bridge_damage = bridge_dmg.get_output_dataset('result')
bridge_damge_df = bridge_damage.get_dataframe_from_csv()

In [ ]:
inventory_df = PlotUtil.inventory_to_geodataframe(bridge_inventory)
inventory_df = PlotUtil.remove_null_inventories(inventory_df, 'guid')
inventory_df = PlotUtil.merge_inventory_w_dmg(inventory_df, bridge_damge_df)
inventory_df.head()

In [ ]:
new_map = PlotUtil.create_geo_map(inventory_df, key='hazardval')
new_map

In [ ]:
new_map = PlotUtil.create_geo_map(inventory_df, key='ls-slight')
new_map

### chaining with bridge damage to calculate mean damage

In [ ]:
md = MeanDamage(client)
md.set_input_dataset("damage", bridge_damage)
md.load_remote_input_dataset("dmg_ratios", "5a284f2cc7d30d13bc081f96")
md.set_parameter("result_name", "bridge_mean_damage")
md.set_parameter("damage_interval_keys",
                 ["none", "ds-slight", "ds-moderat", "ds-extensi",
                  "ds-complet"])
md.set_parameter("num_cpu", 1)
md.run_analysis()

### mean damage histogram

In [ ]:
mean_damage_dataset = md.get_output_dataset('result')
fig = PlotUtil.mean_damage_histogram(mean_damage_dataset, histogram_bins=30)